# Percentile calculation
We need:
+ the sorted list of all frames 
+ the limits of percentile 80 and the 8 groups as index
+ the boarder values of percentile 80 and the 8 groups

Preferably this is done with no additional random sorting of frames with identical success scores, since that would be redundant considering that we later split goals with a certain success score to all groups that contain some frames with that success score


# Over and under the 80th percentile

# If we do not want to randomize (cause Martin Stoller did not and it is redundant if we calculate a goal share as described in MA Stoller):

In [ ]:
# We create a dictionary with one array for each dataset (area & interval)
# These arrays contain each frame (as a list) entailing:
# Success score, home (-3) or away (-1), string filename, minute and second, the half and the time as a string in a practical format
# The array for every dataset is sorted by Success score

import os
import csv
import random
import pickle

        
#1:
game_data = (r'game_data')

# import the datasets
with open("Pickles/Datasets.pkl", "rb") as l:
    datasets = pickle.load(l)


final_dict_nR ={}                                               # create an empty dictionary

for i in range(len(datasets)):
    final_dict_nR[datasets[i]] = []                              # datasets as keys

for dataset in datasets:
    for dir in os.listdir(game_data):
        loc = os.path.join(game_data, dir)
        # os.chdir(loc)                                         # setting the working directory to data folder causes chaos with checkpoints
        for file in os.listdir(loc):
            if file.startswith('zero') & file.endswith("postprocessed.csv"):
                if dataset in file:                                # Check if its a file for the current dataset (area & interval length)
                    with open(os.path.join(loc, file), 'r') as f:
                        reader = csv.reader(f, delimiter =',')
                        for row in reader:
                            if row != []:
                                final_dict_nR[dataset].append([float(row[-1]), -1, file, row[0], row[1]])     # index of filename needs to be adapted later
                                final_dict_nR[dataset].append([float(row[-3]), -3, file, row[0], row[1]])     # content of dictionary see above (1.)
                                                                                                        # (-3 = home, -1 = away),
                                
for dataset in datasets:
    for i in range(len(final_dict_nR[dataset])):
        if 'HZ1' in final_dict_nR[dataset][i][2]:                    # appending a half time indicator 
            final_dict_nR[dataset][i].append(float(1))               # if HZ1 is in file name we append a one
        elif 'HZ2' in final_dict_nR[dataset][i][2]:                  # if HZ2 is in file name 
            final_dict_nR[dataset][i].append(float(2))               # we append a 2
        else:
            final_dict_nR[dataset][i].append('Error')                # if none we append Error (which we use to check later)
        
        if len(str(final_dict_nR[dataset][i][4])) == 1:              # if the seconds is just one digit (less than 10s) we need
            proxy_s = str(0) + str(final_dict_nR[dataset][i][4])     # to add a 0 to get a format that allows us to compare with SOCCER output
        else:
            proxy_s = final_dict_nR[dataset][i][4]
            
        if final_dict_nR[dataset][i][5] == 1:                        # if it is the first half we just combine minutes and seconds
            final_dict_nR[dataset][i].append(".".join([final_dict_nR[dataset][i][3], proxy_s]))   # into one variable
        elif final_dict_nR[dataset][i][5] == 2:                      # if it is the second half we add 45 to the minutes
            proxy_m = str(int(final_dict_nR[dataset][i][3]) + 45)      # and then combine them the same way
            final_dict_nR[dataset][i].append(".".join([proxy_m, proxy_s]))
             
    final_dict_nR[dataset].sort()               # sort by success score primarily and the random order secondly 


In [ ]:
for dataset in datasets:
    print('Error' in final_dict_nR[dataset])                         # checking if halftime indicator worked without Errors

In [ ]:
#2.2 get average success score
for dataset in datasets:
    success_scores = []   
    for datapoint in final_dict_nR[dataset]:
        success_scores.append(datapoint[0])
    print('Average for ' + dataset + ': ' + str(sum(success_scores)/len(success_scores)))
    #print(len(success_scores))

In [ ]:
# numpy alternative to get percentile border values
# Saving the whole bunch to use the 80th percentile values later

import numpy as np

percentiles_10 = {}
for dataset in datasets:
    percentiles_10[dataset] = []
    values = []
    for line in final_dict_nR[dataset]:
        values.append(line[0])
    percentiles_10[dataset] = np.percentile(values, [0,10,20,30,40,50,60,70,80,90,100])          # saving the boarder values in a dicitonary


with open("Pickles/Boarder_values_10.pkl", "wb") as dict:                # saving the boarder values in a dicitonary
    pickle.dump(percentiles_10, dict)
        
percentiles_10

In [ ]:
# splitting the dictionary into over and under 80 resulting in twice as many lists as datasets (dataset_over, dataset_under)
# simultaneously storing limit indexes 

import math
split_dict_nR = {}                           # dicitonary where we split the datasets in over and under the 80 --> 2 arrays (over and under) per dataset (area & interval)
limits_80 = {}                               # simultaneously storing the limits (index) as well as the total length for use later
for dataset in datasets:
    limits_80[dataset] = []
    length = (len(final_dict_nR[dataset]))           # the 80th percentile (when sorted) is just the borderline of 80% of the values
    print(length)
    percentile = length * 0.8                  # get 80% length of the values
    #print(percentile)
    upper =round(percentile)          # substract 1 since index counting starts with zero and add one because the last one is not included :5 --> 1,2,3,4
    print(upper)
    lower = round(percentile)          # upper and lower are the same index as the last index is not included when selecting :5 --> 1,2,3,4
    print(lower)
    limits_80[dataset].append(upper)       # append the limit index
    limits_80[dataset].append(length)      # and the total length
    split_dict_nR["_".join([dataset, 'under'])] = final_dict_nR[dataset][:upper]
    split_dict_nR["_".join([dataset, 'over'])] = final_dict_nR[dataset][lower:]
    
with open("Pickles/index_limits_80.pkl", "wb") as dict:
    pickle.dump(limits_80, dict)

limits_80

In [ ]:
split_dict_nR.keys()

In [ ]:
len(split_dict_nR['30m_0010s_over'])

print(split_dict_nR['30m_0010s_over'][0])
print(split_dict_nR['30m_0010s_over'][-1])

In [ ]:
print(len(split_dict_nR['30m_0010s_under']))

print(split_dict_nR['30m_0010s_under'][0])
print(split_dict_nR['30m_0010s_under'][50000])
print(split_dict_nR['30m_0010s_under'][75000])
print(split_dict_nR['30m_0010s_under'][-1])

# 8 groups &rarr; percentiles

In [ ]:
import numpy as np
import pandas as pd

**length** return the **number of cases** which is **one more** than the last index since 0 is the index of the first case. But since index :x return everything up to x but **excluding x** the length should be the upper limit of the percentile 87.5 - 100%

# if we do not want to randomize we chose this one

In [ ]:
# split into the 8 groups --> 8 lists (by percentiles) per dataset (area & interval)
# saving the index limits and the success score boarder values

import math

percentiles = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
split8_dict_nR = {}
limits_8 = {}
boarder_values_8 = {}
lower_lim = 0
for dataset in datasets:
    limits_8[dataset] = []
    boarder_values_8[dataset] = []
    lower_lim = 0
    length = (len(final_dict_nR[dataset]))           # the 80th percentile (when sorted) is just the borderline of 80% of the values
    for p in range(len(percentiles)):
        percentile = round(length * percentiles[p])            # get percentile, round uses symetrical rounding (.5 --> up or down to go to even number (3.5 --> 4, 2.5 --> 2))
        upper_lim = percentile                                 # this becomes the upper limit
        limits_8[dataset].append(upper_lim)
        boarder_values_8[dataset].append(final_dict_nR[dataset][upper_lim-1][0])
        print(str(lower_lim) +' - ' +  str(upper_lim))         # just to check we print our intervals
        
        split8_dict_nR["_".join([dataset, str(percentiles[p])])] = final_dict_nR[dataset][lower_lim:upper_lim]     # and then split by these intervals
#         if p == 7:                         
#             lower_lim = 0                                      # see below except if its the last round in this loop then we need to start with 0 next time
#         else:
#             lower_lim = upper_lim                              # for the next round in this loop the upper limit of this round become the lower limit for the next
        lower_lim = upper_lim                                    # solved it more elegantly with settting lower_lim in the loop on layer above
limits_8 

with open("Pickles/index_limits_8.pkl", "wb") as dict:
    pickle.dump(limits_8, dict)

with open("Pickles/Boarder_values_8.pkl", "wb") as dict:
    pickle.dump(boarder_values_8, dict)

In [ ]:
boarder_values_8

In [ ]:
split8_dict_nR.keys()

In [ ]:
for key in split8_dict_nR.keys():
    print(key)
    print(len(split8_dict_nR[key]))     # the subsets of each dataset should have the same length which is rouned to 1/8th
# see next cell

In [ ]:
for elem in final_dict_nR:
    print(len(final_dict_nR[elem]))
    print(len(final_dict_nR[elem]) / 8)
    
# this is how long the subsets should be

In [ ]:
# minimum, maximum and most importantly mean success score for every percentile group of every dataset
# means stored as csv

import pandas as pd
percentiles = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
dfs_nR = {}
means_dict = {}
    
for data in split8_dict_nR:                          # for every subset (8 x #datasets)
    dfs_nR[data] = pd.DataFrame(split8_dict_nR[data], columns=['Success Score', 'Home_away', 'csv_file', 'min', 'sec', 'half', 'time_weird_format'])  # creating a dictionary of dataframes
    minimum = dfs_nR[data]['Success Score'].min()
    maximum = dfs_nR[data]['Success Score'].max()
    mean = dfs_nR[data]['Success Score'].mean()
    means_dict[data] = mean
    print(data + ';' + str(minimum) + '-' +str(maximum) + ';' +str(mean))

means_df = pd.DataFrame()
   
for dataset in datasets:
    mean_v = []
    for i in percentiles:
        mean_v.append(means_dict["_".join([dataset, str(i)])])
    means_df[dataset] = mean_v
    

means_df.to_csv('Results/Mean_SuccessScores_8.csv')    
means_df



In [ ]:
import pickle
    
with open("Pickles/80_20_split.pkl", "wb") as cdict:
    pickle.dump(split_dict_nR, cdict)
    


In [ ]:
import pickle

with open("Pickles/AllDatapoints.pkl", "wb") as cdict:
    pickle.dump(final_dict_nR, cdict)

In [ ]:
import pickle 


with open("Pickles/8_split.pkl", "wb") as cdict:
    pickle.dump(split8_dict_nR, cdict)